In [ ]:
from data_loader import load_maps, load_power_ups, load_characters

map_data = load_maps()
power_ups = load_power_ups()
characters = load_characters()

In [ ]:
#other packages
import pandas as pd

# RFC predicts on non-linear relationships
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# n_estimators is # of decision trees to train;
# min_sample_split is num of samples in leaf before node is split
# higher num, less accuracy
rf = RandomForestClassifier(n_estimators = 50, min_samples_split = 10, random_state = 1)


In [ ]:
# add col for both players in race


In [ ]:
# set training set to first 900 rows
train = results.iloc[1:900]

In [ ]:
# test set to last 100 rows
test = results.iloc[900:1000]

In [ ]:
# set predictors: baseline, map performance, power ups etc
predictors = ["baseline_perf","map_perf","powerups","times_hit"]

In [ ]:
# rf is using given predictors to guess "target" whether player won
rf.fit(train[predictors], train["win_stat"])

In [ ]:
# predicting
preds = rf.predict(test[predictors])

In [ ]:
# check overall accuracy (model's overall correctness)
acc = accuracy_score(test["target", preds])

In [ ]:
# check precision (model correctness when predicting specific category)
combined = pd.DataFrame(dict(actual=test["target"], predict=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])
precision_score(test["target"], preds)

In [ ]:
# improve precision by computing rolling averages
grouped_matches = results.groupby("player1")

# ex: peach AND bowser 
group = grouped_matches.get_group("Peach")

In [ ]:
# compute rolling averages based on previous matches between two characters
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("index") # not sure if we'll use index
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group.dropna(subset=new_cols)
    return group

In [ ]:
# columns to compute rolling averages for
cols = ["baseline","map_perf","map","powerups","times_hit"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
# added info about what happened before
rolling_averages(group, cols, new_cols)

In [ ]:
# creates df based on players in race
matches_rolling = results.groupby("player1").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling = matches_rolling.droplevel("playing1")
matches_rolling.index - range(matches_rolling.shape[0])

In [ ]:
# retrain model
def make_predictions(data, predictors):
    train = results.iloc[1:900]
    test = results.iloc[900:1000]
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"],predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision